In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from optuna import Trial
import optuna
from optuna.samplers import TPESampler
from sklearn.model_selection import train_test_split
import xgboost as xgb

plt.rcParams["font.family"] = "Malgun Gothic"  # Windows의 맑은 고딕 폰트 사용
plt.rcParams['axes.unicode_minus'] = False  # 한글 사용 시, 마이너스 기호 깨짐 방지

In [2]:
data = pd.read_csv('train.csv')
print(data.head())

data =  data.drop(columns=["ID"])
print(data.head())

test_data = pd.read_csv('test.csv')
print(test_data.head())

test_data =  test_data.drop(columns=["ID"])
print(test_data.head())

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [21]:
missing_count = data.isnull().sum()
print(missing_count)

제조사            0
모델             0
차량상태           0
배터리용량       2711
구동방식           0
주행거리(km)       0
보증기간(년)        0
사고이력           0
연식(년)          0
가격(백만원)        0
dtype: int64


결측치 처리가 핵심이다 ㅇㅇ

결측치 확인

결측치 처리를 위한 배터리 용량 분석

상관계수 분석
-> 결측치를 처리하기 위해서 가장 관련있는 값들은 무엇인가?




In [22]:
average_warranty_by_model = data.groupby('모델')['보증기간(년)'].mean().sort_values(ascending=True)

# 결과 출력
print("모델별 평균 보증기간:")
print(average_warranty_by_model)

모델별 평균 보증기간:
모델
TayCT     1.764179
Tay       1.808864
TayGTS    1.816000
IONIQ     1.978571
i5        3.570048
i3        3.659794
eT        4.807388
MY        4.837370
ID4       4.867769
MX        4.878788
MS        4.920578
RSeTGT    4.922078
M3        4.960573
Q4eT      4.989418
Niro      5.396985
Soul      5.554156
ION6      6.773087
ION5      6.852691
KNE       6.876712
iX        7.534060
EV6       9.476965
Name: 보증기간(년), dtype: float64


In [23]:
# 배터리 용량 결측치 처리
upper7 = data[data['보증기간(년)']>=7]['배터리용량'].mean()
lower7 = data[data['보증기간(년)']<7]['배터리용량'].mean()

data.fillna(-1, inplace=True)
test_data.fillna(-1, inplace=True)
def fill_battery(row):
    if row['배터리용량'] == -1:
        if row['보증기간(년)'] >= 7:
            return upper7
        else:
            return lower7
    return row['배터리용량']

data['배터리용량'] = data.apply(fill_battery, axis=1)
test_data['배터리용량'] = test_data.apply(fill_battery, axis=1)

missing_count = data.isnull().sum()
print(missing_count)
print("구분선")
missing_count = test_data.isnull().sum()
print(missing_count)


제조사         0
모델          0
차량상태        0
배터리용량       0
구동방식        0
주행거리(km)    0
보증기간(년)     0
사고이력        0
연식(년)       0
가격(백만원)     0
dtype: int64
구분선
제조사         0
모델          0
차량상태        0
배터리용량       0
구동방식        0
주행거리(km)    0
보증기간(년)     0
사고이력        0
연식(년)       0
dtype: int64


In [24]:

data['배터리용량'] = data.groupby('모델')['배터리용량'].transform(lambda x: x.fillna(x.mean()))
print(data[['모델', '배터리용량']].drop_duplicates())
print(data.isnull().sum())

# 트레인 데이터의 모델별 평균 계산
model_avg_battery_capacity = data.groupby('모델')['배터리용량'].mean()

# 테스트 데이터를 트레인 데이터의 평균으로 처리
test_data['배터리용량'] = test_data.apply(
    lambda row: model_avg_battery_capacity[row['모델']]
    if pd.isnull(row['배터리용량']) and row['모델'] in model_avg_battery_capacity
    else row['배터리용량'],
    axis=1
)


          모델      배터리용량
0     TayGTS  86.077000
1       Niro  56.000000
2         eT  91.200000
3     RSeTGT  63.932581
4         i5  61.018000
...      ...        ...
7312   IONIQ  46.090000
7372   IONIQ  56.410000
7379   IONIQ  49.970000
7459   IONIQ  72.960000
7487   IONIQ  67.170000

[267 rows x 2 columns]
제조사         0
모델          0
차량상태        0
배터리용량       0
구동방식        0
주행거리(km)    0
보증기간(년)     0
사고이력        0
연식(년)       0
가격(백만원)     0
dtype: int64


# 제조사별 평균 가격 -> 굳이 건들필요 없다.

In [25]:
# target_encoding
# 범주형 데이터가 가격과 같은 타겟변수와 상관관계가 높은 경우 사용

manufacturer_target_mean = data.groupby('제조사')['가격(백만원)'].mean()
data['제조사'] = data['제조사'].map(manufacturer_target_mean)

test_data['제조사'] = test_data['제조사'].map(manufacturer_target_mean)

# 차량 모델별 평균 가격 -> 굳이 건들 필요 없어 보임

In [26]:
car_avg_price = data.groupby('모델')['가격(백만원)'].mean().sort_values(ascending=True)

price_categories_car = {}

# 차량 모델과 인덱스를 추출
for idx, price in enumerate(car_avg_price.items()):
    price_categories_car[idx] = [price[0]]  # 모델이 키, 가격이 값

print(price_categories_car)

# 범주 데이터 -> 수치 변환
model_to_weight_car = {model: weight for weight, models in price_categories_car.items() for model in models}
# 기존 'vehicle' 열의 데이터를 가격 범주(정수 값)으로 덮어쓰기

{0: ['IONIQ'], 1: ['Soul'], 2: ['i3'], 3: ['KNE'], 4: ['Niro'], 5: ['ION5'], 6: ['ION6'], 7: ['ID4'], 8: ['EV6'], 9: ['M3'], 10: ['Q4eT'], 11: ['i5'], 12: ['eT'], 13: ['MY'], 14: ['MS'], 15: ['iX'], 16: ['MX'], 17: ['RSeTGT'], 18: ['Tay'], 19: ['TayCT'], 20: ['TayGTS']}


In [27]:
grouped_avg_price = data.groupby(['모델', '구동방식'])['가격(백만원)'].mean().reset_index()

price_categories_mech = {
    1: ["FWD"], # 전륜 구동
    2: ["RWD"], #  후륜 구동
    3: ["AWD"] # 4륜 구동
}
# 범주 데이터 -> 수치 변환
model_to_weight_mech = {model: weight for weight, models in price_categories_mech.items() for model in models}
# 기존 'vehicle' 열의 데이터를 가격 범주(정수 값)으로 덮어쓰기

In [28]:
used_avg_price = data.groupby('차량상태')['가격(백만원)'].mean().sort_values(ascending=True)

# 차종별 차량상태에 따른 평균 가격 계산
grouped_avg_price = data.groupby(['모델', '차량상태'])['가격(백만원)'].mean().reset_index()

price_categories_used = {
    1: ["Pre-Owned"], # 중고
    2: ["Nearly New"], #  거의 새것
    3: ["Brand New"] # 새 것
}
# 범주 데이터 -> 수치 변환
model_to_weight_used = {model: weight for weight, models in price_categories_used.items() for model in models}
# 기존 'vehicle' 열의 데이터를 가격 범주(정수 값)으로 덮어쓰기

In [29]:
accident_avg_price = data.groupby('사고이력')['가격(백만원)'].mean().sort_values(ascending=True)

price_categories_accident = {
    0: ["Yes"], # 사고 유무 o
    1 : ["No"], # 사고 유무 X
}

model_to_weight_accident = {model: weight for weight, models in price_categories_accident.items() for model in models}

In [30]:
# 차종별 평균 배터리 용량 계산

average_battery_capacity = data.groupby('모델')['배터리용량'].mean().sort_values(ascending=True)

# Normalize '배터리용량' in data
data['배터리용량'] = data.apply(
    lambda row: row['배터리용량'] / average_battery_capacity[row['모델']],
    axis=1
)

# Normalize '배터리용량' in test_data using the same average_battery_capacity from data
test_data['배터리용량'] = test_data.apply(
    lambda row: row['배터리용량'] / average_battery_capacity[row['모델']]
    if row['모델'] in average_battery_capacity.index else row['배터리용량'],
    axis=1
)


# 데이터를 공통된 Feature로 처리하려고 하면 잘 처리가 안된다.

In [31]:
# 제조사 매핑
# print("----- 제조사 가격 데이터 모델링 -----")
# data["제조사"] = data["제조사"].map(model_to_weight_maker)

#차량 모델 매핑
print("----- 차량 가격 데이터 모델링 -----")
data["모델"] = data["모델"].map(model_to_weight_car)

# 구동 방식 매핑 (컬럼명 수정: '구동방식 ' → '구동방식')
print("----- 구동방식 데이터 모델링 -----")
data["구동방식"] = data["구동방식"].map(model_to_weight_mech)

# 차량 상태 매핑
print("----- 차량 상태 데이터 모델링 -----")
data["차량상태"] = data["차량상태"].map(model_to_weight_used)

# 사고 이력 매핑
print("----- 사고 이력 데이터 모델링 -----")
data["사고이력"] = data["사고이력"].map(model_to_weight_accident)

# 데이터 정보 출력 확인
print("----- 데이터 변환 확인 -----")
print(data.head())
print("결측치 확인:")
print(data.isnull().sum())


print("# -----------------------------------------------------------------")

# print("----- 제조사 가격 데이터 모델링 -----")
# test_data["제조사"] = test_data["제조사"].map(model_to_weight_maker)

#차량 모델 매핑
print("----- 차량 가격 데이터 모델링 -----")
test_data["모델"] = test_data["모델"].map(model_to_weight_car)

# 구동 방식 매핑 (컬럼명 수정: '구동방식 ' → '구동방식')
print("----- 구동방식 데이터 모델링 -----")
test_data["구동방식"] = test_data["구동방식"].map(model_to_weight_mech)

# 차량 상태 매핑
print("----- 차량 상태 데이터 모델링 -----")
test_data["차량상태"] = test_data["차량상태"].map(model_to_weight_used)

# 사고 이력 매핑
print("----- 사고 이력 데이터 모델링 -----")
test_data["사고이력"] = test_data["사고이력"].map(model_to_weight_accident)

# 데이터 정보 출력 확인
print("----- 데이터 변환 확인 -----")
print(test_data.head())
print("결측치 확인:")

----- 차량 가격 데이터 모델링 -----
----- 구동방식 데이터 모델링 -----
----- 차량 상태 데이터 모델링 -----
----- 사고 이력 데이터 모델링 -----
----- 데이터 변환 확인 -----
          제조사  모델  차량상태     배터리용량  구동방식  주행거리(km)  보증기간(년)  사고이력  연식(년)  \
0  131.990934  20     2  1.142461     3     13642        0     1      2   
1   30.779029   4     2  0.954530     1     10199        6     1      0   
2   75.128354  12     3  1.124451     3      2361        7     1      0   
3   75.128354  17     2  0.836865     3     21683        3     1      0   
4   55.212344  11     1  0.880878     3    178205        1     1      0   

   가격(백만원)  
0   159.66  
1    28.01  
2    66.27  
3    99.16  
4    62.02  
결측치 확인:
제조사         0
모델          0
차량상태        0
배터리용량       0
구동방식        0
주행거리(km)    0
보증기간(년)     0
사고이력        0
연식(년)       0
가격(백만원)     0
dtype: int64
# -----------------------------------------------------------------
----- 차량 가격 데이터 모델링 -----
----- 구동방식 데이터 모델링 -----
----- 차량 상태 데이터 모델링 -----
----- 사고 이력 데이터 모델링 -----
----- 데이터 변환 확

In [32]:
# 다중 공선성 처리 -> 그렇게 도움이 되지는 않았다.

# # 두 열을 float 타입으로 변환 후 연산 수행
# data['제조사_모델'] = data['제조사'].astype(float) + data['모델'].astype(float)
# test_data['제조사_모델'] = test_data['제조사'].astype(float) + test_data['모델'].astype(float)
#
# # 기존 열 제거
# data = data.drop(columns=['제조사', '모델'])
# test_data = test_data.drop(columns=['제조사', '모델'])
#
# # 결과 출력
# print(data.head())
# print(test_data.head())


In [33]:
print("Missing values in data after imputation:\n", data.isnull().sum())
print("\nMissing values in test_data after imputation:\n", test_data.isnull().sum())

Missing values in data after imputation:
 제조사         0
모델          0
차량상태        0
배터리용량       0
구동방식        0
주행거리(km)    0
보증기간(년)     0
사고이력        0
연식(년)       0
가격(백만원)     0
dtype: int64

Missing values in test_data after imputation:
 제조사         0
모델          0
차량상태        0
배터리용량       0
구동방식        0
주행거리(km)    0
보증기간(년)     0
사고이력        0
연식(년)       0
dtype: int64


In [34]:
print("결측치 확인:")
final_data_verification = data
final_data_verification.to_csv('final_data_LGM_verification.csv', index=False, encoding='utf-8')

print(final_data_verification.head())

결측치 확인:
          제조사  모델  차량상태     배터리용량  구동방식  주행거리(km)  보증기간(년)  사고이력  연식(년)  \
0  131.990934  20     2  1.142461     3     13642        0     1      2   
1   30.779029   4     2  0.954530     1     10199        6     1      0   
2   75.128354  12     3  1.124451     3      2361        7     1      0   
3   75.128354  17     2  0.836865     3     21683        3     1      0   
4   55.212344  11     1  0.880878     3    178205        1     1      0   

   가격(백만원)  
0   159.66  
1    28.01  
2    66.27  
3    99.16  
4    62.02  


In [35]:
train_data_df =  data.drop(columns=['가격(백만원)'])
target_data_df = data['가격(백만원)']


In [36]:
from sklearn.model_selection import KFold
from sklearn.metrics import root_mean_squared_error


import os
# Optuna에서 최적화할 objective 함수
def objectiveLR_xgb(trial, train_df, target_df):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 7),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True)
    }

    # K-Fold 교차 검증 설정
    n_splits = 100# K-Fold 개수
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    rmse_list = []  # 각 Fold의 RMSE 저장

    # K-Fold 학습
    for train_idx, val_idx in kf.split(train_df):
        X_train, X_val = train_df.iloc[train_idx], train_df.iloc[val_idx]
        y_train, y_val = target_df.iloc[train_idx], target_df.iloc[val_idx]

        # XGBoost 모델 학습
        model = xgb.XGBRegressor(objective='reg:squarederror', **params)
        model.fit(X_train, y_train)

        # 검증 데이터 예측
        y_pred = model.predict(X_val)
        rmse = root_mean_squared_error(y_val, y_pred)
        rmse_list.append(rmse)

    # K-Fold 평균 RMSE 반환
    return np.mean(rmse_list)


# Optuna 환경 설정
study_name = "dacon_temp_xgb_label_target_encode_kfold"
storage_name = f"sqlite:///optuna/{study_name}.db"

if not os.path.exists("optuna"):
    os.makedirs("optuna")

try:
    # 스터디 생성
    study_xgb = optuna.create_study(storage=storage_name,
                                    study_name=study_name,
                                    direction='minimize',
                                    sampler=optuna.samplers.TPESampler(multivariate=True, n_startup_trials=100,
                                                                       seed=42))
    print("create")

except:
    # 기존 스터디 로드
    study_xgb = optuna.load_study(study_name=study_name,
                                  storage=storage_name)
    print("load")

# Optuna 최적화 실행 (Cross Validation 포함)
study_xgb.optimize(lambda trial: objectiveLR_xgb(trial, train_data_df, target_data_df), n_trials=50)


C:\Users\kzxx1\miniconda3\envs\Minnong_pytorch_project\Lib\site-packages\optuna\_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


load


[I 2025-01-13 17:23:53,154] Trial 644 finished with value: 1.3384932091970732 and parameters: {'n_estimators': 168, 'max_depth': 7, 'learning_rate': 0.047370572829449765, 'min_child_weight': 2, 'subsample': 0.8087476943233188, 'colsample_bytree': 0.9090189728449318, 'gamma': 3.700898636870677e-08}. Best is trial 595 with value: 1.3214510122168115.
[W 2025-01-13 17:23:57,042] Trial 645 failed with parameters: {'n_estimators': 163, 'max_depth': 7, 'learning_rate': 0.06028087409996673, 'min_child_weight': 2, 'subsample': 0.793287673696272, 'colsample_bytree': 0.8922382128865163, 'gamma': 5.479944800224603e-08} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\Users\kzxx1\miniconda3\envs\Minnong_pytorch_project\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\kzxx1\AppData\Local\Temp\ipykernel_36256\3071234822.py", line 64, in <lambda>
 

KeyboardInterrupt: 

In [37]:
# 최적 하이퍼파라미터 출력
print("Best Params:", study_xgb.best_params)
print("Best Value (RMSE):", study_xgb.best_value)

# 최적 하이퍼파라미터로 전체 학습 및 예측
best_params = study_xgb.best_params
final_model = xgb.XGBRegressor(objective='reg:squarederror', **best_params)
final_model.fit(train_data_df, target_data_df)

# 테스트 데이터 예측
submit_test_df = test_data  # 테스트 데이터
y_submit = final_model.predict(submit_test_df)

# 제출 파일 생성
submit_ID_df = pd.read_csv('test.csv')[['ID']]  # 테스트 파일 ID 열 가져오기
submit_df = pd.DataFrame(y_submit, columns=['가격(백만원)'])
final_submit_df = pd.concat([submit_ID_df, submit_df], axis=1)
final_submit_df.to_csv('submit_xgb_kfold_이게_맞나.csv', index=False)

print("최종 제출 파일 예시:\n", final_submit_df.head())

Best Params: {'n_estimators': 183, 'max_depth': 7, 'learning_rate': 0.04286291804802631, 'min_child_weight': 2, 'subsample': 0.8418206958076482, 'colsample_bytree': 0.8847331357786226, 'gamma': 5.732052656515655e-08}
Best Value (RMSE): 1.3214510122168115
최종 제출 파일 예시:
          ID     가격(백만원)
0  TEST_000  130.647324
1  TEST_001   80.010452
2  TEST_002   65.133949
3  TEST_003   34.953239
4  TEST_004   47.917049
